In [29]:
import os
import numpy as np
import time
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
seed = 5693
np.random.seed(seed)
tf.random.set_seed(seed)
NUM_OF_CATEGORY = 10

In [30]:
mnist = tf.keras.datasets.fashion_mnist
# mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_final_test, y_final_test = x_test, y_test

x_test = x_train[50000:60000]
x_train = x_train[0:50000]
y_test = y_train[50000:60000]
y_train = y_train[0:50000]

x_train, x_test, x_final_test = np.reshape(x_train / 255.0, (int(tf.shape(x_train)[0]),-1)), np.reshape(x_test / 255.0, (int(tf.shape(x_test)[0]),-1)), np.reshape(x_final_test / 255.0, (int(tf.shape(x_final_test)[0]),-1))
one_hot_y = lambda t: [1 if i == t else 0 for i in range(NUM_OF_CATEGORY)]
y_train, y_test, y_final_test = np.array([one_hot_y(y) for y in y_train]), np.array([one_hot_y(y) for y in y_test]), np.array([one_hot_y(y) for y in y_final_test])


In [31]:
class MLP(tf.keras.Model):
    def __init__(self, size_input, size_hidden, size_output, device=None):
        super(MLP, self).__init__()
        """
        size_input: int, size of input layer
        size_hidden: int, size of hidden layer
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
        """
        self.size_input, self.size_hidden, self.size_output, self.device =\
        size_input, size_hidden, size_output, device
        # Initialize weights between input layer and hidden layer 1
        self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden], seed=seed, stddev=0.1))
        # Initialize biases for hidden layer 1
        self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=seed))
         # Initialize weights between hidden layer 1 and hidden layer 2
        self.W2 = tf.Variable(tf.random.normal([self.size_hidden, self.size_hidden], seed=seed, stddev=0.1))
        # Initialize biases for hidden layer 2
        self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=seed))
         # Initialize weights between hidden layer 2 and output layer
        self.W3 = tf.Variable(tf.random.normal([self.size_hidden, self.size_output], seed=seed, stddev=0.1))
        # Initialize biases for output layer
        self.b3 = tf.Variable(tf.random.normal([1, self.size_output], seed=seed))

        # Define variables to be updated during backpropagation
        self.MLP_variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
        # self.MLP_variables = [self.W1, self.W3, self.b1, self.b3]

    
    def forward(self, X, dropout_rate=0):
        """
        forward pass
        X: Tensor, inputs
        """
        if self.device is not None:
            with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
                self.y = self.compute_output(X, dropout_rate=dropout_rate)
        else:
            self.y = self.compute_output(X, dropout_rate=dropout_rate)

        return self.y
  
    def loss(self, y_pred, y_true, L1=0, L2=0):
        '''
        y_pred - Tensor of shape (batch_size, size_output)
        y_true - Tensor of shape (batch_size, size_output)
        '''
        y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
        y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
        l2_penlty = (tf.nn.l2_loss(self.W1)+tf.nn.l2_loss(self.W2)+tf.nn.l2_loss(self.W3))*L2
        loss_with_l2 = l2_penlty+tf.losses.categorical_crossentropy(y_true_tf, y_pred_tf)
#         print("y_true_tf",y_true_tf,"y_pred_tf",y_pred_tf)
        return loss_with_l2
  
    def backward(self, X_train, y_train, dropout_rate, learning_rate, L1=0, L2=0):
        """
        backward pass
        """
#         optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train,dropout_rate=dropout_rate)
            current_loss = self.loss(predicted, y_train, L2=L2)
        grads = tape.gradient(current_loss, self.MLP_variables)
        
        
        optimizer.apply_gradients(zip(grads, self.MLP_variables))
        return current_loss, predicted
        
        
    def compute_output(self, X, dropout_rate=0):
        """
        Custom method to obtain output tensor during forward pass
        """
        # Cast X to float32
        X_tf = tf.cast(X, dtype=tf.float32)
        #Remember to normalize your dataset before moving forward
        # Compute values in hidden layer 1
        what = tf.matmul(X_tf, self.W1) + self.b1
        hhat = tf.nn.relu(what)

        # Compute values in hidden layer 2
        what = tf.matmul(hhat, self.W2) + self.b2
        hhat = tf.nn.relu(what)

        # Compute output
        output = tf.matmul(hhat, self.W3) + self.b3
        #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
        #Second add tf.Softmax(output) and then return this variable
        return tf.nn.softmax(output)
#         return output


In [32]:
result_for_ten = []
for s in range(10):
    print(s,"!!!!!!!!!!!!!!!!!")
    DROPOUT_RATE = 0
    BATCH_SIZE = 200
    HIDDEN_SIZE = 128
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.001
    L2_PENLTY = 0

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    path = f"OPT-SGD_SEED_{s}-LearnRate_{LEARNING_RATE}-L2_{L2_PENLTY}-TIME_{current_time}"

#     Set log summary

#     train_log_dir = 'logs/mnist/' + path + '/train'
#     test_log_dir = 'logs/mnist/' + path + '/test'
    
    train_log_dir = 'logs/mnist_fashion/' + path + '/train'
    test_log_dir = 'logs/mnist_fashion/' + path + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)


    size_input = int(tf.shape(x_train)[1])
    size_hidden = HIDDEN_SIZE
    size_output = int(tf.shape(y_train)[1])
    number_of_train_examples = int(tf.shape(x_train)[0])
    number_of_test_examples = int(tf.shape(x_test)[0])


    # print("size_input",size_input)
    # print("size_output",size_output)
    # print("number_of_train_examples",number_of_train_examples)
    # print("number_of_test_examples",number_of_test_examples)


    mlp_on_gpu = MLP(size_input, size_hidden, size_output, device='cpu')
    time_start = time.time()
    epoch = 1
    loss_diff,last_loss = 1,0

    while epoch <= NUM_EPOCHS and abs(loss_diff) > 0.00001:
        loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
        train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BATCH_SIZE+BATCH_SIZE//4, seed=epoch*(seed)).batch(BATCH_SIZE)
        for inputs, outputs in train_ds:
            cur_loss, preds = mlp_on_gpu.backward(inputs, outputs, dropout_rate=DROPOUT_RATE, learning_rate=LEARNING_RATE, L2=L2_PENLTY)
            loss_total_gpu += cur_loss
      # Calculate Accuracy
        train_accuracy, test_accuracy = tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.CategoricalAccuracy()
        train_accuracy.update_state(y_train, mlp_on_gpu.forward(x_train))
        test_accuracy.update_state(y_test, mlp_on_gpu.forward(x_test))
        train_loss = np.sum(loss_total_gpu) / x_train.shape[0]
        test_loss = np.sum(mlp_on_gpu.loss(mlp_on_gpu.forward(x_test), y_test)) / x_test.shape[0]

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss, step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss, step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        
        
        loss_diff = train_loss - last_loss
        last_loss = train_loss
        print(f'Number of Epoch = {epoch} - Training Cross Entropy:= {np.sum(loss_total_gpu) / x_train.shape[0]} - Training Accuracy:= {train_accuracy.result().numpy()} - Test Accuracy:= {test_accuracy.result().numpy()}')
        time_taken = time.time() - time_start
        print('Time taken (in seconds): {:.2f}'.format(time_taken))
        time_start = time.time()
        epoch += 1

    # record loss and accuracy for final test set:
    final_test_loss = np.sum(mlp_on_gpu.loss(mlp_on_gpu.forward(x_final_test), y_final_test)) / x_final_test.shape[0]
    final_test_acc = tf.keras.metrics.CategoricalAccuracy()
    final_test_acc.update_state(y_final_test, mlp_on_gpu.forward(x_final_test))
    result_for_ten.append([final_test_loss,final_test_acc.result().numpy()])
#     print(result_for_ten)
    
    

0 !!!!!!!!!!!!!!!!!
Number of Epoch = 1 - Training Cross Entropy:= 0.718595390625 - Training Accuracy:= 0.7893600463867188 - Test Accuracy:= 0.7744000554084778
Time taken (in seconds): 4.85
Number of Epoch = 2 - Training Cross Entropy:= 0.4738335546875 - Training Accuracy:= 0.8456800580024719 - Test Accuracy:= 0.8383000493049622
Time taken (in seconds): 4.83
Number of Epoch = 3 - Training Cross Entropy:= 0.41609703125 - Training Accuracy:= 0.8266600370407104 - Test Accuracy:= 0.8170000314712524
Time taken (in seconds): 4.78
Number of Epoch = 4 - Training Cross Entropy:= 0.3875084375 - Training Accuracy:= 0.8645800352096558 - Test Accuracy:= 0.8548000454902649
Time taken (in seconds): 4.77
Number of Epoch = 5 - Training Cross Entropy:= 0.3639165625 - Training Accuracy:= 0.8766600489616394 - Test Accuracy:= 0.8593000173568726
Time taken (in seconds): 4.76
Number of Epoch = 6 - Training Cross Entropy:= 0.3447345703125 - Training Accuracy:= 0.884320080280304 - Test Accuracy:= 0.86840003728

Number of Epoch = 9 - Training Cross Entropy:= 0.3045353125 - Training Accuracy:= 0.889500081539154 - Test Accuracy:= 0.872700035572052
Time taken (in seconds): 4.78
Number of Epoch = 10 - Training Cross Entropy:= 0.2948971484375 - Training Accuracy:= 0.8964800834655762 - Test Accuracy:= 0.8777000308036804
Time taken (in seconds): 4.76
Number of Epoch = 11 - Training Cross Entropy:= 0.2859942578125 - Training Accuracy:= 0.8897400498390198 - Test Accuracy:= 0.8706000447273254
Time taken (in seconds): 4.75
Number of Epoch = 12 - Training Cross Entropy:= 0.280936328125 - Training Accuracy:= 0.9061600565910339 - Test Accuracy:= 0.8822000622749329
Time taken (in seconds): 4.81
Number of Epoch = 13 - Training Cross Entropy:= 0.270243671875 - Training Accuracy:= 0.8918200731277466 - Test Accuracy:= 0.8715000152587891
Time taken (in seconds): 4.78
Number of Epoch = 14 - Training Cross Entropy:= 0.26210486328125 - Training Accuracy:= 0.9068800806999207 - Test Accuracy:= 0.8808000683784485
Time 

Number of Epoch = 17 - Training Cross Entropy:= 0.2473540234375 - Training Accuracy:= 0.9145000576972961 - Test Accuracy:= 0.8814000487327576
Time taken (in seconds): 4.87
Number of Epoch = 18 - Training Cross Entropy:= 0.2402275 - Training Accuracy:= 0.9096800684928894 - Test Accuracy:= 0.879300057888031
Time taken (in seconds): 4.83
Number of Epoch = 19 - Training Cross Entropy:= 0.23500037109375 - Training Accuracy:= 0.9104000329971313 - Test Accuracy:= 0.8756000399589539
Time taken (in seconds): 4.78
Number of Epoch = 20 - Training Cross Entropy:= 0.232000390625 - Training Accuracy:= 0.9123600721359253 - Test Accuracy:= 0.8824000358581543
Time taken (in seconds): 4.77
5 !!!!!!!!!!!!!!!!!
Number of Epoch = 1 - Training Cross Entropy:= 0.733555859375 - Training Accuracy:= 0.7699400782585144 - Test Accuracy:= 0.7611000537872314
Time taken (in seconds): 4.81
Number of Epoch = 2 - Training Cross Entropy:= 0.472545078125 - Training Accuracy:= 0.8401200771331787 - Test Accuracy:= 0.827900

Number of Epoch = 5 - Training Cross Entropy:= 0.3642973046875 - Training Accuracy:= 0.8672800660133362 - Test Accuracy:= 0.8543000221252441
Time taken (in seconds): 4.85
Number of Epoch = 6 - Training Cross Entropy:= 0.3465478125 - Training Accuracy:= 0.877500057220459 - Test Accuracy:= 0.8613000512123108
Time taken (in seconds): 4.93
Number of Epoch = 7 - Training Cross Entropy:= 0.335847890625 - Training Accuracy:= 0.8828600645065308 - Test Accuracy:= 0.8645000457763672
Time taken (in seconds): 4.82
Number of Epoch = 8 - Training Cross Entropy:= 0.320928203125 - Training Accuracy:= 0.8786600828170776 - Test Accuracy:= 0.8619000315666199
Time taken (in seconds): 4.76
Number of Epoch = 9 - Training Cross Entropy:= 0.3101147265625 - Training Accuracy:= 0.8862400650978088 - Test Accuracy:= 0.8693000674247742
Time taken (in seconds): 4.76
Number of Epoch = 10 - Training Cross Entropy:= 0.30047701171875 - Training Accuracy:= 0.8926400542259216 - Test Accuracy:= 0.8720000386238098
Time tak

Number of Epoch = 13 - Training Cross Entropy:= 0.27589044921875 - Training Accuracy:= 0.895020067691803 - Test Accuracy:= 0.8714000582695007
Time taken (in seconds): 4.76
Number of Epoch = 14 - Training Cross Entropy:= 0.2660304296875 - Training Accuracy:= 0.9050400853157043 - Test Accuracy:= 0.8794000148773193
Time taken (in seconds): 4.78
Number of Epoch = 15 - Training Cross Entropy:= 0.2647605859375 - Training Accuracy:= 0.9073800444602966 - Test Accuracy:= 0.8785000443458557
Time taken (in seconds): 4.78
Number of Epoch = 16 - Training Cross Entropy:= 0.25492248046875 - Training Accuracy:= 0.9077800512313843 - Test Accuracy:= 0.879800021648407
Time taken (in seconds): 4.89
Number of Epoch = 17 - Training Cross Entropy:= 0.250303125 - Training Accuracy:= 0.9130600690841675 - Test Accuracy:= 0.8818000555038452
Time taken (in seconds): 4.82
Number of Epoch = 18 - Training Cross Entropy:= 0.242825078125 - Training Accuracy:= 0.90774005651474 - Test Accuracy:= 0.8755000233650208
Time 

In [35]:
result_for_ten

[[0.3426852783203125, 0.87970006],
 [0.3362916259765625, 0.88210005],
 [0.356945751953125, 0.8744],
 [0.3422876220703125, 0.8798],
 [0.3577452392578125, 0.87280005],
 [0.3520983154296875, 0.87490004],
 [0.3532803466796875, 0.8758],
 [0.36703505859375, 0.8726],
 [0.348924951171875, 0.87600005],
 [0.3668726318359375, 0.87140006]]

In [36]:
print(sum(map(lambda x: x[0], result_for_ten))/len(result_for_ten),
      sum(map(lambda x: x[1], result_for_ten))/len(result_for_ten))

0.3524166821289062 0.875950038433075
